## Finding Maximum

First before implementing anything, we need to import the stuff we need

In [15]:
# importing QISKit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, QISKitError, QuantumJob
from qiskit import available_backends, execute, register, get_backend, compile

from qiskit.tools import visualization
from qiskit.tools.visualization import circuit_drawer

to find maximum value, first we need comparator. To make a comparator, multiple-bit-control gate is needed. We implement the n-bit-control gate here.

In [18]:
def n_control(circuit, q_array, target, ancillary_array, flip_array=None):
    # error checking
    if flip_array is not None and len(q_array) != len(flip_array):
        raise ValueError("q_array(len:{}) and flip_array(len:{}) must have the same length".format(len(q_array), len(flip_array)))
    if len(ancillary_array) < len(q_array) - 2:
        raise ValueError("ancillary array length is not enough ({}) for q_array ({})".format(len(ancillary_array), len(q_array)))
    if flip_array is None:
        flip_array = [1 for _ in range(len(q_array))]
    # todo handle wrong flip_array
    
    n = len(q_array)
    
    # put X-gate if flip
    for i in range(n):
        if flip_array[i] == -1:
            circuit.x(q_array[i])
    
    circuit.ccx(q_array[0], q_array[1], ancillary_array[0])
    for i in range(2, n-1):
        circuit.ccx(q_array[i], ancillary_array[i-2], ancillary_array[i-1])
    circuit.ccx(q_array[n-1], ancillary_array[n-3], target)
    for i in reversed(range(2, n-1)):
        circuit.ccx(q_array[i], ancillary_array[i-2], ancillary_array[i-1])
    circuit.ccx(q_array[0], q_array[1], ancillary_array[0])
    
    # clean up X-gate
    for i in range(n):
        if flip_array[i] == -1:
            circuit.x(q_array[i])

Let's see how the n_control circuit works by looking at 4-bit control gate where second and third bit should be flipped.